In [4]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

import os
import random

path_name = '/home/anna/datasets/Bluecoat/'
preprocessing_path = path_name + 'preprocess/'
openDNS_path = path_name + 'opendns-fetchstats-master/'

## STEP 1: Pre-processing
(1) wget ~55G of Syrian Bluecoat Proxy logs
    * This was only from specific files from SG-42 (all others were redacted completely)
    * Perform randomized selection of ~1 million records for analysis
    * Logs from SG-42 are provided in the zip file below
(2) Preprocessing Stage 1:
    * Remove malformed records
    * Remove records that prevent reading into DataFrame
(3) Preprocessing Stage 2:
    * Identify logs that have sufficient data (for example, cs-host, c-ip, csUserAgent)
(4) Preprocessing Stage 3:
    * Obtain OpenDNS web categorization
    * Enrich Bluecoat dataset
(5) Preprocessing Stage 4:
    * Conversion of text fields to numeric to support matrix form
    

### Processing Stage 1:  Prepare data so it can be read into Data Frame

In [119]:
# These two lines are needed else it will not display all columns (26 total in my dataframe)
from IPython.display import display
pd.options.display.max_columns = None
# These two lines resize the plot area and font size
plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['font.size'] = 14

In [59]:
# Directly doing a pandas read_csv doesn't work because the data is not perfect.  Needs to be cleaned
# before going into a PD dataframe.
n = 0
bad_idx = []
lines = []
with open(preprocessing_path + 'sgos422_sampled_set.csv', 'r') as f:
    reader = csv.reader(f, delimiter=' ')
    #reader = pd.read_csv(f, delimiter=' ')
    for row in reader:
        # Needed to suppress an extra blank space placed between fields 9 and 10
        # csReferer and sc-status
        row = filter(lambda x: x != '', row)
        lines.append(row)
        if len(row) != 25:
            bad_idx.append(n)
        n += 1

In [60]:
# Out of 2.04 mill records, just 9 bad rows 
bad_idx

[24669, 335866, 335867, 335868, 671733, 1357468, 1703679, 1703680, 2039986]

In [61]:
# Combination of corrupted lines and device restarts (comment lines)
for i in bad_idx:
    print lines[i]

['2011-07-22', '20:38:25', '1554', 'd521fbde365a0e33', '-', '-', '-', 'OBSERVED', 'unavailable', 'http://chitchat.mybrowserbar.com/cgi/errors.cgi?q=http://chitchat.mybrowserbar.com/cgi/errors.cgi%3fq%3dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%253fq%253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%25253fq%25253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%2525253fq%2525253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%252525253fq%252525253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%25252525253fq%25252525253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%2525252525253fq%2525252525253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%252525252525253fq%252525252525253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%25252525252525253fq%25252525252525253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%2525252525252525253fq%2525252525252525253dhttp://chitchat.mybrowserbar.com/cgi/errors.cgi%252525252525252525253fq%252525252525252525253dhttp://chitchat.mybrowserbar.com/cgi/e

In [62]:
# started out with 2,040,000 records, removed 9
for i in range(len(bad_idx)-1,-1,-1):
    del lines[bad_idx[i]]

# Validation: They are now good lines
print len(lines)
for i in bad_idx:
    print lines[i]

2039991
['2011-07-22', '20:38:25', '52', '547ee6aa6e9c53a2', '-', '-', '-', 'OBSERVED', 'unavailable', 'http://ads.fling.com/www/delivery/afr.php?zoneid=410', '200', 'TCP_HIT', 'GET', 'text/css', 'http', 'ads.fling.com', '80', '/www/delivery/lib/fe/410/rk_footer_t4.css', '-', 'css', 'Mozilla/5.0 (Windows NT 5.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1', '82.137.200.42', '1291', '1199', '-']
['2011-07-22', '21:25:35', '1430', '8d1782d7bb752977', '-', '-', '-', 'OBSERVED', 'unavailable', 'http://www.4shared.com/video/bjhYhata/_-_.htm', '200', 'TCP_NC_MISS', 'GET', '-', 'http', 'www.ltassrv.com', '80', '/track/default.aspx', '?p=56570&v=5&s=35448&c=25129&cc=GB&t=1&r=&o=&a=&pv=0&rnd=5624&as=0', 'aspx', 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SIMBAR={FCA9BE86-0442-11D6-8418-0013D44BF6EA}; .NET CLR 2.0.50727; MSN Optimized;US)', '82.137.200.42', '242', '436', '-']
['2011-07-22', '21:25:35', '203', '03b470633854e2f1', '-', '-', '-', 'OBSERVED', 'unavailable', 'http://s

In [63]:
# Validate lines again
n = 0
bad_idx = []
for l in lines:
    l = filter(lambda x: x!= '', row)
    if len(l) != 25:
        bad_idx.append(n)
    n += 1

In [64]:
len(bad_idx)

0

In [66]:
# Example of good well-formed record
print lines[0]
print len(lines)

['2011-07-22', '20:34:51', '282', 'ce6de14af68ce198', '-', '-', '-', 'OBSERVED', 'unavailable', 'http://www.surfjunky.com/members/sj-a.php?r=44864', '200', 'TCP_NC_MISS', 'GET', 'text/html', 'http', 'www.surfjunky.com', '80', '/members/sj-a.php', '?r=66556', 'php', 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.65 Safari/534.24', '82.137.200.42', '1395', '663', '-']
2039991


### Processing Stage 2:  Further cull data
* Remove row that do not have critical data such as:
  - c_ip
  - cs-host

In [43]:
# Read into pandas dataframe
cols = ['dt', 'tm', 'time_taken', 'c_ip', 'cs_username', 'cs_auth_group', 'x_exception_id', 'sc_filter_result', 'cs_categories', 'csReferer', 'sc_status', 's_action', 'cs_method', 'rsContentType', 'cs_uri_scheme', 'cs_host','cs_uri_port', 'cs_uri_path', 'cs_uri_query', 'cs_uri_extension', 'csUserAgent', 's_ip', 'sc_bytes', 'cs_bytes', 'x_virus_id']
sgos = pd.read_csv(path_name + 'sgos422_sampled_clean.csv', names=cols, sep= ',')

In [39]:
sgos.head(5)
#sgos.tail(5)

,dt,tm,time_taken,c_ip,cs_username,cs_auth_group,x_exception_id,sc_filter_result,cs_categories,csReferer,...,cs_host,cs_uri_port,cs_uri_path,cs_uri_query,cs_uri_extension,csUserAgent,s_ip,sc_bytes,cs_bytes,x_virus_id
0,2011-07-22,20:34:51,282,ce6de14af68ce198,-,-,-,OBSERVED,unavailable,http://www.surfjunky.com/members/sj-a.php?r=44864,...,www.surfjunky.com,80,/members/sj-a.php,?r=66556,php,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,82.137.200.42,1395,663,-
1,2011-07-22,20:34:51,216,6154d919f8d56690,-,-,-,OBSERVED,unavailable,http://x31.iloveim.com/build_3.9.2.1/comet.html,...,x31.iloveim.com,80,/servlets/events,?1122064400327,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US...,82.137.200.42,473,1129,-
2,2011-07-22,20:34:51,102,48fd5c69d3cfc0af,-,-,-,OBSERVED,unavailable,http://static.xhamster.com/xplayer17.swf,...,88.208.24.131,80,"/key=io8g3-zl3cM,end=1311337549/data=480312379...",-,flv,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,82.137.200.42,724,319,-
3,2011-07-22,20:34:51,716,f46e16fe0221b453,-,-,-,OBSERVED,unavailable,-,...,media2.lsops.net,80,/idle/1894876757/425,-,-,Shockwave Flash,82.137.200.42,182,160,-
4,2011-07-22,20:34:51,615,96ba5993c403a175,-,-,-,OBSERVED,unavailable,http://ifa.camads.net/dif/?cid=xvideos-shared-...,...,feeds.videosz.com,80,/custom/xvideos/464x244.php,-,php,Mozilla/5.0 (Windows NT 5.1; rv:2.0) Gecko/201...,82.137.200.42,378,474,-


In [40]:
sgos.shape

(1964138, 25)

In [41]:
# cs-host '-' typically due to: 
#   internal_error: The CacheFlow appliance encountered an unexpected error that resulted in the inability to handle the current transaction.
#   invalid_request: CacheFlow appliance was unable to handle the received request because the syntax of the request is incorrect.
#   unsupported_protocol: The protocol used in the request is not understood.
#   policy_denied: miniscule # of rows with little information
dash_host = sgos[sgos.cs_host == '-']
print len(dash_host)
pd.options.display.max_columns = None
dash_host.x_exception_id.value_counts()

0


Series([], Name: x_exception_id, dtype: int64)

In [73]:
bad_idx = [ k for k, row in sgos.iterrows() if sgos.loc[k, 'cs_host'] in ['-', '', None] ]

In [51]:
# Not much information to be gleaned also it is 10 rows out of 2M (or .0005%)
dash_host[(dash_host.x_exception_id == 'policy_denied') & (sgos.cs_host == '-')]

,dt,tm,time_taken,c_ip,cs_username,cs_auth_group,x_exception_id,sc_filter_result,cs_categories,csReferer,sc_status,s_action,cs_method,rsContentType,cs_uri_scheme,cs_host,cs_uri_port,cs_uri_path,cs_uri_query,cs_uri_extension,csUserAgent,s_ip,sc_bytes,cs_bytes,x_virus_id
33682,2011-07-22,20:39:41,120048,322a2ab2f2081d8c,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,68,-
145602,2011-07-22,20:56:15,827,11a3939f1df15f15,-,-,policy_denied,DENIED,unavailable,-,403,TCP_NC_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,780,506,-
145667,2011-07-22,20:56:16,165,11a3939f1df15f15,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,192,-
263541,2011-07-22,21:14:30,120003,4c483c502eabbfa9,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,68,-
908388,2011-07-22,22:58:46,120032,a3305f247d74ff1a,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,68,-
1174202,2011-07-22,23:45:18,60632,723fbffabf41b202,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,100,-
1386983,2011-07-23,07:44:22,1719,723fbffabf41b202,-,-,policy_denied,DENIED,unavailable,-,403,TCP_NC_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,780,602,-
1764096,2011-07-23,14:53:56,119989,318280433c811b4f,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,68,-
1777162,2011-07-23,14:55:43,19816,888ad1f19577b536,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,68,-
1817447,2011-07-23,15:01:21,120298,8ef3c586ba275fb9,-,-,policy_denied,DENIED,unavailable,-,0,TCP_ERR_MISS,unknown,-,-,-,0,/,-,-,-,82.137.200.42,0,68,-


In [74]:
print len(bad_idx)
print sgos.loc[bad_idx[75852], 'cs_host']
print len(lines)

75853
-
2039991


In [75]:
for i in range(len(bad_idx)-1,-1,-1):
    del lines[bad_idx[i]]

In [89]:
print len(sgos)
print len(lines)
bad_idx = sgos[ (sgos.cs_host == '-') | (sgos.cs_host == '') | (sgos.cs_host == None) ]
print len(bad_idx)

1964138
1964138
0


In [78]:
# Use these two fields to enrich the data frame
urls = sgos[['csReferer', 'cs_host']]

In [79]:
len(urls)

1964138

In [80]:
urls.head(10)

,csReferer,cs_host
0,http://www.surfjunky.com/members/sj-a.php?r=44864,www.surfjunky.com
1,http://x31.iloveim.com/build_3.9.2.1/comet.html,x31.iloveim.com
2,http://static.xhamster.com/xplayer17.swf,88.208.24.131
3,-,media2.lsops.net
4,http://ifa.camads.net/dif/?cid=xvideos-shared-...,feeds.videosz.com
5,-,porn194.xvideos.com
6,http://videogayz.com/,99.192.176.43
7,http://x32.iloveim.com/servlets/ajax,x32.iloveim.com
8,http://www.sham-sat.net/vb/showthread.php?t=63170,www.sham-sat.net
9,http://static.xhamster.com/xplayer17.swf,88.208.24.132


In [90]:
print urls.loc[1, 'csReferer']
print urls.loc[1, 'cs_host']

http://x31.iloveim.com/build_3.9.2.1/comet.html
x31.iloveim.com


In [91]:
# Check to see if there are nulls in csReferer field or cs_host
dash_vals = urls[ (urls.cs_host == '-') | (urls.cs_host == '') | (urls.cs_host == None) ] 
#null_vals = [urls.loc[i] for i in range(0, len(sgos)) if (urls.loc[i, 'cs_host'] in (None, '')) ]       

In [92]:
len(dash_vals)

0

In [94]:
len(urls.cs_host.value_counts())

21146

In [95]:
len(urls.csReferer.value_counts())

201789

### Processing Stage 3: Enrich data
* Obtain categorization by using OpenDNS 
* The script below was run on a separate VM on an isolated machine 
* Please note that a few of the domains were identified as malware and botnet, execute at your own risk!
* For approximately 2M get requests took several hours (as a python script outside of Notebook)


 ### Use OpenDNS FetchStats to get Domain Categories

In [34]:
categories_only = ['Academic_Fraud','Adult_Themes','Adware','Alcohol','Anime_Manga_Webcomic','Auctions',
    'Automotive','Blogs','Business_Services','Chat','Classifieds','Dating','Drugs',
    'Ecommerce_Shopping','Educational_Institutions','File_Storage','Financial_Institutions',
    'Forums_Message_boards','Gambling','Games','German_Youth_Protection','Government',
    'Hate_Discrimination','Health_and_Fitness','Humor','Instant_Messaging','Jobs_Employment',
    'Lingerie_Bikini','Movies','Music','News_Media','Non-Profits','Nudity','P2P_File_sharing',
    'Parked_Domains','Photo_Sharing','Podcasts','Politics','Pornography','Portals',
    'Proxy_Anonymizer','Radio','Religious','Research_Reference','Search_Engines','Sexuality',
    'Social_Networking','Software_Technology','Sports','Tasteless','Television','Tobacco',
    'Travel','Video_Sharing','Visual_Search_Engines','Weapons','Web_Spam','Webmail']
opendns_col = [
    'Rank','Domain','Total','Blacklisted','Blocked_by_Category','Blocked_as_Botnet',
    'Blocked_as_Malware','Blocked_as_Phishing','Resolved_by_SmartCache']

opendns_col.extend(categories_only)
print len(opendns_col)
webcat = pd.read_csv(openDNS_path + '2016-01-26_openDNS_fetchstats.csv', sep = ',', names= opendns_col, header = 0)

67


In [35]:
webcat.head(3)

,Rank,Domain,Total,Blacklisted,Blocked_by_Category,Blocked_as_Botnet,Blocked_as_Malware,Blocked_as_Phishing,Resolved_by_SmartCache,Academic_Fraud,...,Sports,Tasteless,Television,Tobacco,Travel,Video_Sharing,Visual_Search_Engines,Weapons,Web_Spam,Webmail
0,1,www.google.com,1919,0,1919,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,*.youtube.com,1590,0,1590,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,caldav.calendar.yahoo.com,852,0,852,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
del webcat['Rank']
del webcat['Total']
del webcat['Blocked_by_Category']
del webcat['Blocked_as_Botnet']
del webcat['Blocked_as_Malware']
del webcat['Blocked_as_Phishing']
del webcat['Resolved_by_SmartCache']
del webcat['Blacklisted']

In [37]:
webcat.head(3)

,Domain,Academic_Fraud,Adult_Themes,Adware,Alcohol,Anime_Manga_Webcomic,Auctions,Automotive,Blogs,Business_Services,...,Sports,Tasteless,Television,Tobacco,Travel,Video_Sharing,Visual_Search_Engines,Weapons,Web_Spam,Webmail
0,www.google.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,*.youtube.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,caldav.calendar.yahoo.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
cols = ['dt', 'tm', 'time_taken', 'c_ip', 'cs_username', 'cs_auth_group', 'x_exception_id', 'sc_filter_result', 'cs_categories', 'csReferer', 'sc_status', 's_action', 'cs_method', 'rsContentType', 'cs_uri_scheme', 'Domain','cs_uri_port', 'cs_uri_path', 'cs_uri_query', 'cs_uri_extension', 'csUserAgent', 's_ip', 'sc_bytes', 'cs_bytes', 'x_virus_id']
sgos = pd.read_csv(path_name + 'sgos422_sampled_clean.csv', names=cols, sep= ',')

In [50]:
sgos.shape

(1964138, 25)

In [52]:
sgos_cat = sgos.merge(webcat, how='left', on='Domain')

In [53]:
sgos_cat.head(3)

,dt,tm,time_taken,c_ip,cs_username,cs_auth_group,x_exception_id,sc_filter_result,cs_categories,csReferer,sc_status,s_action,cs_method,rsContentType,cs_uri_scheme,Domain,cs_uri_port,cs_uri_path,cs_uri_query,cs_uri_extension,csUserAgent,s_ip,sc_bytes,cs_bytes,x_virus_id,Academic_Fraud,Adult_Themes,Adware,Alcohol,Anime_Manga_Webcomic,Auctions,Automotive,Blogs,Business_Services,Chat,Classifieds,Dating,Drugs,Ecommerce_Shopping,Educational_Institutions,File_Storage,Financial_Institutions,Forums_Message_boards,Gambling,Games,German_Youth_Protection,Government,Hate_Discrimination,Health_and_Fitness,Humor,Instant_Messaging,Jobs_Employment,Lingerie_Bikini,Movies,Music,News_Media,Non-Profits,Nudity,P2P_File_sharing,Parked_Domains,Photo_Sharing,Podcasts,Politics,Pornography,Portals,Proxy_Anonymizer,Radio,Religious,Research_Reference,Search_Engines,Sexuality,Social_Networking,Software_Technology,Sports,Tasteless,Television,Tobacco,Travel,Video_Sharing,Visual_Search_Engines,Weapons,Web_Spam,Webmail
0,2011-07-22,20:34:51,282,ce6de14af68ce198,-,-,-,OBSERVED,unavailable,http://www.surfjunky.com/members/sj-a.php?r=44864,200,TCP_NC_MISS,GET,text/html,http,www.surfjunky.com,80,/members/sj-a.php,?r=66556,php,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,82.137.200.42,1395,663,-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2011-07-22,20:34:51,216,6154d919f8d56690,-,-,-,OBSERVED,unavailable,http://x31.iloveim.com/build_3.9.2.1/comet.html,200,TCP_NC_MISS,GET,text/html;charset=UTF-8,http,x31.iloveim.com,80,/servlets/events,?1122064400327,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US...,82.137.200.42,473,1129,-,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2011-07-22,20:34:51,102,48fd5c69d3cfc0af,-,-,-,OBSERVED,unavailable,http://static.xhamster.com/xplayer17.swf,403,TCP_NC_MISS,GET,text/html,http,88.208.24.131,80,"/key=io8g3-zl3cM,end=1311337549/data=480312379...",-,flv,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,82.137.200.42,724,319,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
sgos_cat.shape

(1964138, 83)

In [135]:
import math
no_webcat = sgos_cat['Academic_Fraud'].apply(math.isnan)

In [144]:
sgos[no_webcat].Domain.value_counts()

88.208.24.131                                                    85613
88.208.24.132                                                    84316
88.208.24.194                                                    19527
46.229.160.7                                                     10698
88.208.24.196                                                     7596
88.208.24.138                                                     7508
216.245.211.226                                                   6281
194.187.98.230                                                    4882
194.187.98.229                                                    4834
194.187.98.231                                                    4659
173.193.242.218                                                   3433
174.140.154.22                                                    3274
69.162.125.244                                                    2508
46.229.160.162                                                    1988
46.229

In [140]:
print "Number of unique domains not matched: ", len(sgos[no_webcat].Domain.value_counts())
print "Total number of domains not matched: ", len(sgos[no_webcat].Domain)

Number of unique domains not matched:  1687
Total number of domains not matched:  318205


In [143]:
len(sgos.Domain.value_counts())

21146

In [159]:
import socket

def check_ip(addr):
    try:
        socket.inet_aton(addr)
        return True 
    except socket.error:
        return False

In [171]:
vals_nocat = sgos[no_webcat].Domain.value_counts()
ip_addr = pd.Series(vals_nocat.index)
is_ip = ip_addr.apply(check_ip)
print sum(is_ip), 'out of', len(sgos[no_webcat].Domain.value_counts()), 'are IP addresses'

rows_nocat = sgos[no_webcat].Domain
is_ip = rows_nocat.apply(check_ip)
print sum(is_ip), 'out of', len(sgos[no_webcat].Domain), 'are IP addresses'
print 'Or:', str(100*sum(is_ip)/float(len(sgos[no_webcat].Domain))) + '%'


1019 out of 1687 are IP addresses
297793 out of 318205 are IP addresses
Or: 93.5852673591%


### Processing Stage 4: Convert Fields to Numeric 
**Categorical:**
* **dt:** convert to weekday or weekend (categorical)
* **tm:** convert to business hours, after hours, weekend hours
* **sc_filter_result:** observed, proxied, denied
* **sc_status:** 200, and others
* **s_action:** action taken to process request: allowed, denied,
  failed, server_error
* **cs_method:** GETs and a few PUTs 
* **rsContentType:** text/html, jpeg, etc
* **cs_uri_scheme:** http, few https, and other junk 
* **cs_port:** categorical
* **csUserAgent:** what browser or client 
* **c_ip:** ipv6 address in octets (tbd)
* **s_ip:** this is the sgos device ip (always the same)
* **x_virus_id:** not sure if this is ever populated

**Continuous:**
* **time_taken:** ms 
* **sc_bytes:** server to client bytes
* **cs_bytes:** client to server bytes 
* **cs_uri_path:** just get length or skip 
* **cs_uri_query:** see above 
* **cs_uri_extension:** see above 

In [2]:
from datetime import date, datetime
import time

In [5]:
cols = ['dt', 'tm', 'time_taken', 'c_ip', 'cs_username', 'cs_auth_group', 'x_exception_id', 'sc_filter_result', 'cs_categories', 'csReferer', 'sc_status', 's_action', 'cs_method', 'rsContentType', 'cs_uri_scheme', 'Domain','cs_uri_port', 'cs_uri_path', 'cs_uri_query', 'cs_uri_extension', 'csUserAgent', 's_ip', 'sc_bytes', 'cs_bytes', 'x_virus_id']
sg_mat = pd.read_csv(path_name + 'sgos422_sampled_clean.csv', names=cols, sep= ',')

### DT/TM field conversion

In [6]:
# python isoweekday() defines Mon=1, Tue=2, ..., Sun=7
def detmWeekday(dtm):
    dt = time.strptime(dtm, '%Y-%m-%d %H:%M:%S')
    #print dt.tm_wday
    if dt.tm_wday in range(0,5):
        return 1
    else:
        return 0

In [7]:
def detmHours(dtm):
    dt = time.strptime(dtm, '%Y-%m-%d %H:%M:%S')
    #print dt, dt.tm_wday, dt.tm_hour
    return dt.tm_hour
    

In [8]:
detmWeekday('2011-07-22 12:00:00')

1

In [9]:
detmHours('2016-01-31 09:00:00')

9

In [10]:
# Combine date time fields into one for easier processing
sg_mat['dtm'] = sg_mat['dt'] + " " + sg_mat['tm']

In [11]:
sg_mat['weekday'] = sg_mat['dtm'].apply(detmWeekday)
sg_mat['hrs'] = sg_mat['dtm'].apply(detmHours)

In [12]:
print sg_mat.shape
sg_mat.head(3)

(1964138, 28)


,dt,tm,time_taken,c_ip,cs_username,cs_auth_group,x_exception_id,sc_filter_result,cs_categories,csReferer,...,cs_uri_query,cs_uri_extension,csUserAgent,s_ip,sc_bytes,cs_bytes,x_virus_id,dtm,weekday,hrs
0,2011-07-22,20:34:51,282,ce6de14af68ce198,-,-,-,OBSERVED,unavailable,http://www.surfjunky.com/members/sj-a.php?r=44864,...,?r=66556,php,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,82.137.200.42,1395,663,-,2011-07-22 20:34:51,1,20
1,2011-07-22,20:34:51,216,6154d919f8d56690,-,-,-,OBSERVED,unavailable,http://x31.iloveim.com/build_3.9.2.1/comet.html,...,?1122064400327,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US...,82.137.200.42,473,1129,-,2011-07-22 20:34:51,1,20
2,2011-07-22,20:34:51,102,48fd5c69d3cfc0af,-,-,-,OBSERVED,unavailable,http://static.xhamster.com/xplayer17.swf,...,-,flv,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,82.137.200.42,724,319,-,2011-07-22 20:34:51,1,20


### sc_filter_result field conversion

In [13]:
print sg_mat.sc_filter_result.value_counts()

OBSERVED    1932188
DENIED        23512
PROXIED        8438
Name: sc_filter_result, dtype: int64


In [23]:
sc_filter_dummies = pd.get_dummies(sg_mat.sc_filter_result, prefix = 'sc_filter_result')
sc_filter_dummies.head(2)

,sc_filter_result_DENIED,sc_filter_result_OBSERVED,sc_filter_result_PROXIED
0,0,1,0
1,0,1,0


In [25]:
sc_filter_dummies.drop(sc_filter_dummies.columns[2], axis=1, inplace=True)

In [26]:
sc_filter_dummies.head(2)

,sc_filter_result_DENIED,sc_filter_result_OBSERVED
0,0,1
1,0,1


In [28]:
sg_mat = pd.concat([sg_mat, sc_filter_dummies], axis=1)

In [29]:
sg_mat.head(2)

,dt,tm,time_taken,c_ip,cs_username,cs_auth_group,x_exception_id,sc_filter_result,cs_categories,csReferer,...,csUserAgent,s_ip,sc_bytes,cs_bytes,x_virus_id,dtm,weekday,hrs,sc_filter_result_DENIED,sc_filter_result_OBSERVED
0,2011-07-22,20:34:51,282,ce6de14af68ce198,-,-,-,OBSERVED,unavailable,http://www.surfjunky.com/members/sj-a.php?r=44864,...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,82.137.200.42,1395,663,-,2011-07-22 20:34:51,1,20,0,1
1,2011-07-22,20:34:51,216,6154d919f8d56690,-,-,-,OBSERVED,unavailable,http://x31.iloveim.com/build_3.9.2.1/comet.html,...,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US...,82.137.200.42,473,1129,-,2011-07-22 20:34:51,1,20,0,1


### sc_status field (no conversion needed)

In [34]:
print "Value_counts for sc_status:", len(sg_mat.sc_status.value_counts())
print "All records have an sc_status:", sum(sg_mat.sc_status.value_counts())
print "\n", sg_mat.sc_status.value_counts()

Value_counts for sc_status: 31
All records have an sc_status: 1964138

200    1165966
403     265051
304     181508
400     103404
302      81001
206      60876
404      33145
0        22472
503      21791
301      19039
204       6510
416       1306
500       1250
410        316
401        236
509         73
307         42
303         42
406         35
100         25
502         17
504         11
408          6
405          5
501          3
203          3
411          1
300          1
412          1
202          1
201          1
Name: sc_status, dtype: int64


### cs_uri_scheme field conversion

In [35]:
sg_mat.cs_uri_scheme.value_counts()

http    1962254
tcp        1864
rtsp         20
Name: cs_uri_scheme, dtype: int64

In [36]:
uri_scheme_dummies = pd.get_dummies(sg_mat.cs_uri_scheme, prefix = 'cs_uri_scheme')
uri_scheme_dummies.drop(uri_scheme_dummies.columns[2], axis=1, inplace=True)
uri_scheme_dummies.head(2)

,cs_uri_scheme_http,cs_uri_scheme_rtsp
0,1,0
1,1,0


In [39]:
# Something wrong here, need to re-do this one (maybe ran it more than once)
sg_mat = pd.concat([sg_mat, uri_scheme_dummies], axis=1)
sg_mat.head(3)

,dt,tm,time_taken,c_ip,cs_username,cs_auth_group,x_exception_id,sc_filter_result,cs_categories,csReferer,...,weekday,hrs,sc_filter_result_DENIED,sc_filter_result_OBSERVED,cs_uri_scheme_http,cs_uri_scheme_rtsp,cs_uri_scheme_http,cs_uri_scheme_rtsp,cs_uri_scheme_http,cs_uri_scheme_rtsp
0,2011-07-22,20:34:51,282,ce6de14af68ce198,-,-,-,OBSERVED,unavailable,http://www.surfjunky.com/members/sj-a.php?r=44864,...,1,20,0,1,1,0,1,0,1,0
1,2011-07-22,20:34:51,216,6154d919f8d56690,-,-,-,OBSERVED,unavailable,http://x31.iloveim.com/build_3.9.2.1/comet.html,...,1,20,0,1,1,0,1,0,1,0
2,2011-07-22,20:34:51,102,48fd5c69d3cfc0af,-,-,-,OBSERVED,unavailable,http://static.xhamster.com/xplayer17.swf,...,1,20,0,1,1,0,1,0,1,0


### cs_uri_port field (no conversion)

In [307]:
sg_mat.cs_uri_port.value_counts()

80      1962235
443        1845
8080         38
2041          9
81            5
9001          2
1990          1
1935          1
9998          1
84            1
Name: cs_uri_port, dtype: int64

In [16]:
len(sg_mat.s_action.value_counts())

14

In [304]:
sg_mat.cs_method.value_counts()

GET                                                              1814559
POST                                                              145724
CONNECT                                                             1864
HEAD                                                                1325
OPTIONS                                                              414
get                                                                  111
PUT                                                                   88
DESCRIBE                                                              20
PROPFIND                                                              12
l%17���%133���2%10�w�7%1Eۖ��6M|W�%0FD%03%04%05��                       1
!GET                                                                   1
��oN�%08ƶ�aBC���+�VX%0B�s`�@�%14�y��i�%18%0ET_H�'�7x                   1
�E��D%18                                                               1
�����                                              

In [305]:
sg_mat.rsContentType.value_counts()

text/html                                                                                     487752
image/gif                                                                                     256756
image/jpeg                                                                                    246128
text/html;charset=UTF-8                                                                       146013
-                                                                                             103056
image/png                                                                                      75996
text/javascript                                                                                70482
text/html;%20charset=utf-8                                                                     62463
application/x-fcs                                                                              58477
text/html;%20charset=iso-8859-1                                                            

In [310]:
sg_mat.csUserAgent.value_counts()
# https://developer.mozilla.org/en-US/docs/Browser_detection_using_the_user_agent

Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)                                                                                                                                                                                                                                                                                                                                              199225
Mozilla/5.0 (Windows NT 5.1; rv:5.0) Gecko/20100101 Firefox/5.0                                                                                                                                                                                                                                                                                                                                  87822
Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/20100101 Firefox/5.0                                                                                                                                                           

In [312]:
len(sg_mat.c_ip.value_counts())

38017

In [313]:
sg_mat.s_ip.value_counts()

82.137.200.42    1964138
Name: s_ip, dtype: int64

In [314]:
sg_mat.x_virus_id.value_counts()

-    1964138
Name: x_virus_id, dtype: int64